In [1]:
import os

dir = r"data/P02/avro"
participant_id = "P02"
from preprocessors.concat_avro import AvroMerger


avro_files = []
for root, dirs, files in os.walk(dir):
    for file in files:
        if file.endswith(".avro"):
            avro_files.append(os.path.join(root, file))

merged_avro_output_path = os.path.join(dir, f"{participant_id}_merged.avro")

if os.path.exists(merged_avro_output_path):
    print(f"Merged file already exists at: {merged_avro_output_path}. Skipping merge.")
else:
    print(f"No merged file found at: {merged_avro_output_path}. {len(avro_files)} files will be merged.")
    avromerger = AvroMerger(avro_files)
    avromerger.merge_avro_files(merged_avro_output_path)





Merged file already exists at: data/P02/avro\P02_merged.avro. Skipping merge.


In [2]:
from preprocessors.avropreprocessor import AvroAcc
avro_acc = AvroAcc(merged_avro_output_path)


In [3]:
avro_acc.load()

,x_g,y_g,z_g,unixtime_us,datetime
0,1.035645,0.030273,0.070801,1762509333467593,2025-11-07 09:55:33.467593
1,1.029785,0.012207,0.091309,1762509333483218,2025-11-07 09:55:33.483218
2,1.003418,-0.019531,0.151855,1762509333498843,2025-11-07 09:55:33.498843
3,1.011719,-0.031738,0.178223,1762509333514468,2025-11-07 09:55:33.514468
4,0.997559,-0.041504,0.222656,1762509333530093,2025-11-07 09:55:33.530093
...,...,...,...,...,...
1277979,0.039062,-0.025391,1.002930,1762529301791871,2025-11-07 15:28:21.791871
1277980,0.032715,-0.026855,1.005371,1762529301807496,2025-11-07 15:28:21.807496
1277981,0.045410,-0.023926,0.999023,1762529301823121,2025-11-07 15:28:21.823121
1277982,0.040039,-0.018555,0.999023,1762529301838746,2025-11-07 15:28:21.838746


In [4]:
from datetime import datetime, timedelta

# Your timestamps
time_start = "2025-11-07T09:30:00.000Z"
time_end = "2025-11-07T12:30:00.000Z"

def iso_to_micros(iso_string):
    """
    Convert ISO 8601 timestamp string to microseconds since epoch.
    
    Args:
        iso_string: ISO 8601 formatted string (e.g., "2025-11-07T09:30:00.000Z")
    
    Returns:
        Timestamp in microseconds
    """
    dt = datetime.fromisoformat(iso_string.replace('Z', '+00:00'))
    seconds_since_epoch = dt.timestamp()
    micros = int(seconds_since_epoch * 1_000_000)
    return micros

# Convert to microseconds instead of nanoseconds
start_micros = iso_to_micros(time_start)
end_micros = iso_to_micros(time_end)

print(f"Start: {time_start} -> {start_micros} microseconds")
print(f"End:   {time_end} -> {end_micros} microseconds")
print(f"Your first sample: 1762509333467593 microseconds")
print(f"Sample datetime: {datetime.fromtimestamp(1762509333467593 / 1_000_000)}")

# Now trim with microsecond timestamps
trimmed_output_path = os.path.join(dir, f"{participant_id}_trimmed.avro")
avromerger = AvroMerger(avro_files)
avromerger.trim_avro_by_time(merged_avro_output_path, trimmed_output_path, start_micros, end_micros)

Start: 2025-11-07T09:30:00.000Z -> 1762507800000000 microseconds
End:   2025-11-07T12:30:00.000Z -> 1762518600000000 microseconds
Your first sample: 1762509333467593 microseconds
Sample datetime: 2025-11-07 09:55:33.467593

[INFO] Trimming from 1762507800000000 to 1762518600000000
  accelerometer: 1277984 -> 593097 samples
    New timestampStart: 1762509333467593 (2025-11-07 09:55:33.467593)
  eda: 79872 -> 37066 samples
    New timestampStart: 1762509333701968 (2025-11-07 09:55:33.701968)
  temperature: 19968 -> 9266 samples
    New timestampStart: 1762509334451968 (2025-11-07 09:55:34.451968)
  bvp: 1277504 -> 593059 samples
    New timestampStart: 1762509333456636 (2025-11-07 09:55:33.456636)
  systolicPeaks: 28435 -> 0 peaks
  steps: 6646 -> 3088 samples
    New timestampStart: 1762509336454347 (2025-11-07 09:55:36.454347)
[OK] Trimmed Avro file written to: data/P02/avro\P02_trimmed.avro


In [5]:
trimmed_avro_acc = AvroAcc(trimmed_output_path)
trimmed_avro_acc.load()

,x_g,y_g,z_g,unixtime_us,datetime
0,1.035645,0.030273,0.070801,1762509333467593,2025-11-07 09:55:33.467593
1,1.029785,0.012207,0.091309,1762509333483218,2025-11-07 09:55:33.483218
2,1.003418,-0.019531,0.151855,1762509333498843,2025-11-07 09:55:33.498843
3,1.011719,-0.031738,0.178223,1762509333514468,2025-11-07 09:55:33.514468
4,0.997559,-0.041504,0.222656,1762509333530093,2025-11-07 09:55:33.530093
...,...,...,...,...,...
593092,0.001953,0.041992,-1.017090,1762518600484800,2025-11-07 12:30:00.484800
593093,0.009277,0.030762,-1.017090,1762518600500425,2025-11-07 12:30:00.500425
593094,0.001953,0.035645,-1.016602,1762518600516050,2025-11-07 12:30:00.516050
593095,-0.000977,0.029785,-1.015137,1762518600531674,2025-11-07 12:30:00.531674
